In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4
    Uninstalling networkx-3.4:
      Successfully uninstalled networkx-3.4
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [ ]:
!pip install geopy osmnx

In [ ]:
import osmnx as ox
from shapely.geometry import Point  # For creating a point from coordinates
import requests
import pandas as pd

def get_suitable_lands_and_rivers(city):
    """Retrieves large land areas suitable for wind and solar farming and rivers for a given city using the Overpass API."""

    city = city.capitalize()

    # Define the central city area as a polygon or bounding box (for simplicity, this is a placeholder)
    # Replace this with actual coordinates or a bounding box for the central city
    central_city_bounds = None  # Replace with actual bounds if available

    # Overpass API query
    query = f"""
    [out:json];
    area[name="{city}"]->.searchArea;
    (
      way[waterway=river](area.searchArea);
      way[landuse=grass](area.searchArea);
      way[landuse=meadow](area.searchArea);
      way[landuse=park](area.searchArea);
      way[landuse=farmland](area.searchArea);
      way[landuse=forest](area.searchArea);
    );
    out body;
    """

    # Request data from Overpass API
    try:
        response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()

        # Parse the data
        features = []
        for element in data['elements']:
            feature_type = None
            if 'tags' in element:
                if 'waterway' in element['tags']:
                    feature_type = 'river'
                elif 'landuse' in element['tags']:
                    landuse_type = element['tags']['landuse']
                    if landuse_type in ['grass', 'meadow', 'park', 'farmland']:
                        feature_type = 'suitable_land'
                    elif landuse_type == 'forest':
                        feature_type = 'forest_land'

                # Extract the name, defaulting to 'Unnamed' if not available
                name = element['tags'].get('name', 'Unnamed')

                # Extract coordinates if available
                location = None
                if 'lat' in element and 'lon' in element:
                    location = (element['lat'], element['lon'])

                features.append({
                    'type': feature_type,
                    'name': name,
                })

        # Create DataFrame
        res = pd.DataFrame(features)

        # Check if required columns exist
        if 'name' not in res.columns:
            return pd.DataFrame(columns=['type', 'name', 'location'])

        # Convert names to lowercase for case-insensitive comparison
        res['name_lower'] = res['name'].str.lower()

        # Remove rows where name is 'Unnamed'
        filtered_df = res[res['name_lower'] != 'unnamed']

        # Drop duplicate names, keeping only the first occurrence (case-insensitive)
        unique_features_df = filtered_df.drop_duplicates(subset='name_lower')

        # Drop the temporary lowercase name column
        unique_features_df = unique_features_df.drop(columns=['name_lower'])

        # Filter out land areas within the central city bounds (if applicable)
        if central_city_bounds:
            # Implement your logic to exclude areas within the central city bounds
            pass

        # Check if the DataFrame is empty
        if unique_features_df.empty:
            print("No valid data found for the given city.")
            return pd.DataFrame(columns=['type', 'name', 'location'])

        return unique_features_df

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame(columns=['type', 'name', 'location'])


Final

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import google.generativeai as genai

# Load datasets
pollution_df = pd.read_csv('/content/Air pollution Data 1.csv', encoding='ISO-8859-1')
population_df = pd.read_csv('/content/Population data.csv', encoding='ISO-8859-1')
consumption_df = pd.read_csv('/content/Energy_consumption data.csv', encoding='ISO-8859-1')

# Clean data
pollution_df.fillna('Unknown', inplace=True)
population_df.fillna('Unknown', inplace=True)
consumption_df.fillna('Unknown', inplace=True)

# Standardize city names
pollution_df['City'] = pollution_df['City'].str.strip().str.lower()
population_df['City'] = population_df['City'].str.strip().str.lower()
consumption_df['City'] = consumption_df['City'].str.strip().str.lower()

# Merge data
merged_df = pd.merge(pollution_df, population_df, on='City', how='outer')
merged_df = pd.merge(merged_df, consumption_df, on='City', how='outer')

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Function to format data for the model
def format_data_for_model(row):
    return (
        f"City: {row['City']}\n"
        f"SO2 Emission: {row['so2']} gms\n"
        f"NO2 Emission: {row['no2']} gms\n"
        f"SPM Emission: {row['spm']} gms\n"
        f"Population: {row['population_total']}\n"
        f"Total Electricity Consumption: {row['Total Consumption']} kWh\n"
        "Analyze the impact of these pollution levels on health and environment, including health effects by age category."
    )

# Function to generate a report using the model
def generate_report_with_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        inputs['input_ids'],
        max_length=500,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    report = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return report.strip()

# Function to get a report for the city
def get_report_for_city(city):
    city = city.lower()
    city_data = merged_df[merged_df['City'] == city]

    if not city_data.empty:
        prompt = format_data_for_model(city_data.iloc[0])
        report = generate_report_with_model(prompt)
        return report
    else:
        return "There is no excess of emission in this city."

# Retrieve API key for Gemini model
def get_api_key(file_path):
    """Reads the API key from a text file."""
    try:
        with open(file_path, 'r') as file:
            api_key = file.read().strip()
            return api_key
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Path to your text file
file_path = '/content/Api_Key.txt'
GOOGLE_API_KEY = get_api_key(file_path)

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("Failed to retrieve the API key.")

# Initialize Gemini model
model_gemini = genai.GenerativeModel('gemini-pro')

# Function to get power plant data and environmental impact from Gemini
def get_power_plant_data_and_impact(city_report, total_consumption):
    prompt = (
         f"Based on the following report: \n{city_report}\n"
        f"Provide information about the power plants in the {city_name} city, their impact on the environment, and suggest alternative energy source based on the available resources of {city_name} city. Also discuss the optimality of the alternate source with current power source by considering the power usage of {city_name} city."
        f"Based on the following city report:\n{city_report}\n\n"
        f"Considering the total electricity consumption of {total_consumption} kWh for the city,\n\n"
        f"1. **Current Power Stations:**\n"
        f"   - Provide detailed information about the existing power stations in {city_name} city, including their types, capacities, and locations.\n"
        f"   - Assess their current environmental impact, including emissions, resource use, and any negative effects on local ecosystems and communities.\n\n"
        f"2. **Alternative Energy Sources:**\n"
        f"   - Suggest viable alternative energy sources that could potentially replace or supplement the current power stations. Consider both renewable (e.g., solar, wind, hydro, biomass) and non-renewable options (e.g., natural gas).\n"
        f"   - For each suggested alternative, provide an analysis of its feasibility, including resource availability, environmental impact, and suitability for the city’s conditions.\n\n"
        f"3. **Feasibility Assessment:**\n"
        f"   - Determine if the suggested alternatives can meet or exceed the city’s total electricity consumption needs of {total_consumption} kWh.\n"
        f"   - Compare the power generation capacity of each alternative against the current power stations to ensure they can adequately replace or supplement the existing energy sources.\n"
        f"   - Discuss any potential challenges or considerations for transitioning to the suggested alternatives, including costs, infrastructure requirements, and long-term sustainability.\n\n"
        f"4. **Health Effects by Age Category:**\n"
        f"   - Provide detailed information on the health effects of the current power sources and alternative sources on different age groups (e.g., children, adults, elderly).\n\n"
        f"Please provide a comprehensive report that includes all the above points, and conclude with recommendations on the most suitable alternative energy sources for the city, if necessary."
    )

    response = model_gemini.generate_content(prompt)
    result = ''.join([p.text for p in response.candidates[0].content.parts])
    return result

# Function to format dataframe for model
def format_dataframe_for_model(df):
    """Formats the dataframe into a detailed text prompt for the Gemini model."""
    prompt = "Here is a list of locations with their types:\n"
    for _, row in df.iterrows():
        prompt += f"- {row['type'].capitalize()}: {row['name']}\n"
    prompt += "\nBased on the above locations, evaluate the feasibility of implementing power stations (wind and solar) at these locations. Consider factors like land use and environmental impact. Provide suggestions on the optimal type of power station for each location."
    prompt += "\nBased on the above locations, please evaluate the feasibility of implementing wind and solar power stations at these sites. Consider the following factors:\n"
    prompt += "1. **Land Use:** Assess the suitability of each location for power station installation, including land area, current land use, and any restrictions or regulations.\n"
    prompt += "2. **Environmental Impact:** Analyze the potential environmental impact of installing wind or solar power stations at these locations. Consider impacts on local ecosystems, wildlife, and any potential for pollution or resource depletion.\n"
    prompt += "3. **Infrastructure Needs:** Evaluate the infrastructure requirements for each type of power station. This includes access to the grid, transportation for equipment, and any modifications needed to the land.\n"
    prompt += "4. **Optimal Type of Power Station:** Based on the analysis, provide recommendations for the most suitable type of power station (wind or solar) for each location. Justify your recommendations with considerations on efficiency, cost, and environmental benefits.\n"
    prompt += "5. **Additional Considerations:** Mention any other relevant factors that could affect the feasibility of the power stations, such as proximity to existing power infrastructure, potential for future expansion, and community acceptance.\n"
    return prompt

# Function to get feasibility report
def get_feasibility_report(df):
    """Generates a feasibility report for power stations based on the dataframe."""
    prompt = format_dataframe_for_model(df)
    response = model_gemini.generate_content(prompt)
    result = ''.join([p.text for p in response.candidates[0].content.parts])
    return result


<ipython-input-10-35ec6073a7d9>:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pollution_df.fillna('Unknown', inplace=True)
<ipython-input-10-35ec6073a7d9>:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  consumption_df.fillna('Unknown', inplace=True)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in al

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:

# Main interaction loop
print('\nReady to chat...')
while True:
    city_name = input("Enter the city name: ")
    if city_name.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    # Fetch suitable lands and rivers
    unique_features_df = get_suitable_lands_and_rivers(city_name)

    # Get environmental report
    city_report = get_report_for_city(city_name)

    if "no excess of emission" in city_report.lower():
        print(city_report)
    else:
        # Get power plant feasibility
        total_consumption = merged_df.loc[merged_df['City'] == city_name.lower(), 'Total Consumption'].values[0]
        power_plant_info = get_power_plant_data_and_impact(city_report, total_consumption)

        # Get feasibility report based on suitable lands and rivers
        feasibility_report = get_feasibility_report(unique_features_df)

        print(f"City Report:\n{city_report}")
        print(f"Power Plant and Environmental Impact:\n{power_plant_info}")
        print(f"Feasibility Report:\n{feasibility_report}")



Ready to chat...
There is no excess of emission in this city.
There is no excess of emission in this city.
